# INTRODUCTION

In this problem set you will write a solution to an optimization problem on how to find the shortest route from one building to another on the MIT campus given that you wish to constrain the amount of time you will spend walking outdoors (because generally speaking, the nocturnal beaver... er, the nocturnal MIT engineer... hates the sun).

## GETTING STARTED

Download Files: Problem Set 5 skeleton code. This zip archive includes the following files:

* ps5.py: code skeleton
* graph.py: a set of graph-related data structures (Digraph, Node, and Edge) -- use this version of graph.py rather than the one from lecture
* mit_map.txt: a sample data file that holds the information about an MIT campus map.

## BACKGROUND

A graph consists of a set of nodes, also known as vertices, (n<sub>1</sub>, n<sub>2</sub>, n<sub>3</sub>, ....) and a set of edges (e<sub>1</sub>, e<sub>2</sub>, e<sub>3</sub>, ...) where an edge connects two nodes that are in the graph. The node n<sub>1</sub> has children nodes if there exists an edge from n<sub>1</sub> to each child node. In Figure 1, Node "a" has children nodes "b" and "c".

There are two types of edges: directed and undirected. If the edge is directed, then the edge has a specific direction going from start to destination node. Graphs with directed edges are called directed graphs (or digraph).

<img src='ProblemSet5/directedgraph.gif', width='25%'/>
Figure 1. Example of a directed graph where each edge has a specific direction.

If the edge is undirected, also known as bidirectional, then it no longer matters which node is the start or destination node because you can traverse the edge from one node to the other in either direction. Essentially, a link in the graph can be represented by a directed edge going from Node "d" to Node "e" and a directed edge going in the reverse direction.

<img src='ProblemSet5/undirectedgraph.gif', width='25%'/>
Figure 2. Example of an undirected graph where each edge is bidirectional.

An edge can also have a weight. If every edge is associated with a real number (edge weight), then we have weighted graph.

<img src='ProblemSet5/weightedgraph.gif', width='25%'/>
Figure 3. Example of an weighted graph where each edge has a weight associated with it.

In a graph theory problem, the objective function is the function to be minimized (or maximized). For example, choosing the shortest path for airplane flights is an optimization problem where the objective function is to minimize the distance traveled. The nodes are the destination airports and edges are the presence of airplane routes between airports. We can add additional constraints on the problem that must be satisfied such as requiring that the plane only make at most 2 stops along the way from start to end destination. Then the shortest path is only valid if it satisfies the constraint.

## THE MIT MAP

<img src='ProblemSet5/campusmap.gif', width='25%'/>

Here is the map of the MIT campus. From the text input file, mit_map.txt, you will build a representation of this map in Python using the graph-related data structures that we provide.

Each line in mit_map.txt has 4 pieces of data in it in the following order separated by a single space (space-delimited): the start building, the destination building, the distance in meters between the two buildings, and the distance in meters between the two buildings that must be spent outdoors. For example, suppose the map text file contained the following line:

10     32     200     40

This means that the map contains an edge from building 10 (start location) to building 32 (end location) that is 200 meters long, where 40 of those 200 meters are spent outside.

To make the problem interesting, we will say that not every route between a pair of buildings is bi-directional. For example, it may be possible to get from building 54 (Green building) to building 56, but not the other way around, because the wind that blows away from the Green building is too strong.

## Problem 1 - Creating the Data Structure Representation

(20 points possible)<br>
In this problem set, we are dealing with edges that have different weights. In the figure below, the blue numbers with a bar above them show the cost of traversing an edge in terms of total distance traveled, while the green numbers show the cost of traversing an edge in terms of distance spent outdoors. Note that the distance spent outdoors for a single edge is always less than or equal to the total distance it takes to traverse that edge. Now the cost of going from "a" to "b" to "e" is a total distance traveled of 22 meters, where 14 of those meters are spent outdoors. These weights are important when comparing multiple paths because you want to look at the weights associated with the edges in the path instead of just the number of edges traversed.

<img src='ProblemSet5/weightedWithContraint.png', width='25%'/>

In ```graph.py```, you’ll find the ```Digraph```, ```Node```, and ```Edge``` classes, which do not store information about weights associated with each edge.

Extend the classes so that it fits our case of a weighted graph. Think about how you can modify the classes to store the weights shown above. Make modifications directly in graph.py. We highly recommend that you read through the entire problem set before settling on a particular implementation and representation of nodes and edges.

### Hint: Creating subclasses

Subclass the provided classes to add your own functionality to the new classes. Deciding what representation to use in order to build up the graph is the most challenging part of the problem set, so think through the problem carefully. As a start, ```WeightedEdge``` should be a subclass of ```Edge```, and ```WeightedGraph``` should be a subclass of ```Digraph```. 

Define a ```WeightedDigraph``` class to represent your graph. You will also need to define a ```WeightedEdge``` class to represent the edges of your graph. Be sure to use subclassing and inheritance. With your ```WeightedDigraph``` implementation, you should be able to replicate the transcript, which begins to model the graph.

In [3]:
#graph.py

# 6.00.2x Problem Set 5
# Graph optimization
#
# A set of data structures to represent graphs
#

class Node(object):
    def __init__(self, name):
        self.name = str(name)
        
    def getName(self):
        return self.name
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __hash__(self):
        # Override the default hash method
        # Think: Why would we want to do this?
        return self.name.__hash__()

class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
        
    def getSource(self):
        return self.src
    
    def getDestination(self):
        return self.dest
    
    def __str__(self):
        return '{0}->{1}'.format(self.src, self.dest)

class Digraph(object):
    """
    A directed graph
    """
    def __init__(self):
        # A Python Set is basically a list that doesn't allow duplicates.
        # Entries into a set must be hashable (where have we seen this before?)
        # Because it is backed by a hashtable, lookups are O(1) as opposed to the O(n) of a list (nifty!)
        # See http://docs.python.org/2/library/stdtypes.html#set-types-set-frozenset
        self.nodes = set([])
        self.edges = {}
        
    def addNode(self, node):
        if node in self.nodes:
            # Even though self.nodes is a Set, we want to do this to make sure we
            # don't add a duplicate entry for the same node in the self.edges list.
            raise ValueError('Duplicate node')
        else:
            self.nodes.add(node)
            self.edges[node] = []
            
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
        
    def childrenOf(self, node):
        return self.edges[node]
    
    def hasNode(self, node):
        return node in self.nodes
    
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[str(k)]:
                res = '{0}{1}->{2}\n'.format(res, k, d)
        return res[:-1]

In [43]:
class WeightedEdge(Edge):
    def __init__(self, src, dest, distance, outdoor):
        Edge.__init__(self, src, dest)
        self.distance = distance
        self.outdoor = outdoor
    
    def __str__(self):
        return '{0}->{1} ({2}, {3})'.format(self.src, self.dest, self.distance, self.outdoor)
    
    def getTotalDistance(self):
        return self.distance
    
    def getOutdoorDistance(self):
        return self.outdoor


In [95]:
class WeightedDigraph(Digraph):
    """
    A directed graph
    """
    def __init__(self):
        Digraph.__init__(self)
        
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        dist = edge.getTotalDistance()
        outdoor = edge.getOutdoorDistance()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append([dest, (float(dist), float(outdoor))])
    
    def __str__(self):
        res = ''
        for nsource in self.edges:
            for n2 in self.edges[nsource]:
                ndest, weights  = n2
                res = '{0}{1}->{2} ({3}, {4})\n'.format(res, nsource.getName(), ndest.getName(), *weights)
        return res[:-1]
    
    def childrenOf(self, node):
        # children are stored as:
        # node: [[m, (71.0, 44.0)], [h, (30.0, 10.0)], [m, (55.0, 31.0)], [h, (69.0, 46.0)]]
        children = []
        for el in self.edges[node]:
            node, weight = el
            children.append(node)
        return children

In [96]:
# Testing WeightedDigraph and WeightedEdge
g = WeightedDigraph()
na = Node('a')
nb = Node('b')
nc = Node('c')
g.addNode(na)
g.addNode(nb)
g.addNode(nc)
e1 = WeightedEdge(na, nb, 15, 10)
print e1
print e1.getTotalDistance()
print e1.getOutdoorDistance()

e2 = WeightedEdge(na, nc, 14, 6)
e3 = WeightedEdge(nb, nc, 3, 1)
print e2
print e3

g.addEdge(e1)
g.addEdge(e2)
g.addEdge(e3)
print g

a->b (15, 10)
15
10
a->c (14, 6)
b->c (3, 1)
a->b (15.0, 10.0)
a->c (14.0, 6.0)
b->c (3.0, 1.0)


In [98]:
# Test 2
nh = Node('h')
nj = Node('j')
nk = Node('k')
nm = Node('m')
ng = Node('g')
g = WeightedDigraph()
g.addNode(nh)
g.addNode(nj)
g.addNode(nk)
g.addNode(nm)
g.addNode(ng)
randomEdge = WeightedEdge(nk, nm, 90, 87)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nj, nm, 71, 44)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nk, nj, 52, 19)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nj, nh, 30, 10)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nj, nm, 55, 31)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nk, nm, 10, 5)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nh, nj, 56, 23)
g.addEdge(randomEdge)
randomEdge = WeightedEdge(nj, nh, 69, 46)
g.addEdge(randomEdge)

print g.childrenOf(nh)
print g.childrenOf(nj)
print g.childrenOf(nk)
print g.childrenOf(nm)
print g.childrenOf(ng)

[j]
[m, h, m, h]
[m, j, m]
[]
[]


## Problem 2-1

(5 points possible)<br>
Decide how the campus map problem can be modeled as a graph. For your own benefit, we encourage you to write a description of your design approach as a comment under the Problem 2 heading in ps5.py. This exercise will help you guide your thoughts as you approach this problem. Specific questions you may want to consider are: What do the graph's nodes represent in this problem? What do the graph's edges represent in this problem?

```python
# Problem 2:
#
# Before you write any code, write a couple of sentences here 
# describing how you will model this problem as a graph. 

# This is a helpful exercise to help you organize your
# thoughts before you tackle a big design problem!
#
```

In the load_map function of ps5.py read in the building data from mapFilename and build a directed graph to properly represent the MIT campus map (according to the file).

```python
def load_map(mapFilename):
    """
    Parses the map file and constructs a directed graph 

    Parameters:
        mapFilename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four
        positive integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a directed graph representing the map
    """
    # TO DO
```

### Hint: Pseudocode

If you are getting an error "No such file or directory: 'mit_map.txt' ", you will have to prepend the string representing the folder where mit_map.txt is located when you open the file. 

Pseudocode: You will have to create a WeightedGraph object, then iterate line by line in the file. Each line is composed of 4 integers. The first two should be saved as Node objects with a WeightedEdge between them. The tricky part will be adding the nodes and edges to the graph -- you will see that these methods raise exceptions, which you will have to handle with try-except blocks.

When you have implemented this function, instantiate a graph and input your results for the following calls:

```python
>>> mitMap = load_map("mit_map.txt")
>>> print isinstance(mitMap, Digraph)
```

```python
>>> print isinstance(mitMap, WeightedDigraph)
```

In [102]:
mitmap = 'ProblemSet5/mit_map.txt'
#ps5.py

# 6.00.2x Problem Set 5
# Graph optimization
# Finding shortest paths through MIT buildings
#

import string

In [ ]:
#
# Problem 2: Building up the Campus Map
#
# Before you write any code, write a couple of sentences here 
# describing how you will model this problem as a graph. 

# This is a helpful exercise to help you organize your
# thoughts before you tackle a big design problem!
#
# The graph is represented by nodes meaning the buildings and the edges are the paths between buildings.
# As the buildings have no bidirectional way, the paths are represented by a Direct Graph.
# Each edge has two weights, meaning the total distance between buildings and the distance walked outside.
#

In [109]:
def load_map(mapFilename):
    """ 
    Parses the map file and constructs a directed graph

    Parameters: 
        mapFilename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive 
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a directed graph representing the map
    """
    print "Loading map from file..."
    with open(mapFilename) as fin:
        g = WeightedDigraph()
        for line in fin:
            src, dest, dist, out = line.strip().split()
            nsrc = Node(src)
            ndest = Node(dest)
            wedge = WeightedEdge(nsrc, ndest, int(dist), int(out))
            if not g.hasNode(nsrc):
                g.addNode(nsrc)
            if not g.hasNode(ndest):
                g.addNode(ndest)
            g.addEdge(wedge)
    return g
            
mitMap = load_map(mitmap)
print isinstance(mitMap, Digraph)
print isinstance(mitMap, WeightedDigraph)

Loading map from file...
True
True


## Problem 2-2

(5/5 points)<br>
You should be able to obtain a set of which Nodes are in this map. This set should consist of strings. For example, if the mitMap has two nodes, Node('a') and Node('b'), the set of nodes should look like this, when printed out:

set([a, b])

```python
>>> nodes = mitMap.nodes
```

In the box below, define a variable called nodes. Set it equal to the value of mitMap.nodes

In [110]:
nodes = mitMap.nodes
print nodes

set([54, 50, 62, 64, 66, 68, 24, 26, 48, 46, 1, 3, 2, 5, 4, 7, 6, 9, 8, 13, 76, 38, 10, 39, 12, 14, 16, 33, 32, 57, 56, 37, 36, 35, 34, 18, 31])


## Problem 2-3

(5 points possible)<br>
You should be able to obtain a dictionary of which edges are in this map. This dictionary should map a string to a list of edges, which are represented as a list of [string, (total_distance, outdoor_distance)]. For example, if the mitMap has three nodes, a = Node('a'), b = Node('b'), and c = Node('c'), and there are edges: WeightedEdge(a, b, 10, 9), WeightedEdge(a, c, 12, 2), WeightedEdge(b, c, 1, 1), the dictionary of edges should look like this, when printed out:

{a: [[b, (10.0, 9.0)], [c, (12.0, 2.0)]], c: [], b: [[c, (1.0, 1.0)]]}

```python
>>> edges = mitMap.edges
```

In the box below, define a variable called edges. Set it equal to the value you get by calling mitMap.edges.

In [111]:
edges = mitMap.edges
print edges

{54: [[56, (40.0, 30.0)], [66, (45.0, 35.0)], [18, (20.0, 10.0)], [62, (20.0, 10.0)], [14, (70.0, 60.0)], [50, (80.0, 70.0)]], 50: [[14, (50.0, 23.0)], [14, (25.0, 20.0)]], 62: [[54, (20.0, 10.0)], [64, (30.0, 20.0)]], 64: [[62, (30.0, 20.0)]], 66: [[68, (51.0, 0.0)], [56, (40.0, 0.0)], [76, (130.0, 100.0)], [32, (70.0, 60.0)]], 68: [[32, (110.0, 80.0)], [76, (72.0, 30.0)], [66, (51.0, 0.0)], [56, (80.0, 70.0)]], 24: [[13, (35.0, 30.0)], [26, (25.0, 20.0)], [34, (27.0, 0.0)], [12, (33.0, 0.0)]], 26: [[36, (34.0, 0.0)], [16, (45.0, 0.0)], [12, (30.0, 25.0)], [24, (25.0, 20.0)]], 48: [[32, (80.0, 50.0)], [36, (100.0, 80.0)], [46, (25.0, 10.0)]], 46: [[32, (90.0, 40.0)], [36, (80.0, 40.0)], [48, (25.0, 10.0)]], 1: [[2, (75.0, 60.0)], [4, (80.0, 65.0)], [3, (36.0, 0.0)], [5, (32.0, 0.0)]], 3: [[10, (32.0, 0.0)], [4, (60.0, 50.0)], [2, (70.0, 50.0)], [1, (36.0, 0.0)], [7, (25.0, 0.0)]], 2: [[6, (41.0, 0.0)], [14, (51.0, 0.0)], [4, (36.0, 0.0)], [10, (70.0, 50.0)], [3, (70.0, 50.0)], [1, (75

## Problem 3 - Finding the Shortest Path Using Brute Force

(20 points possible)<br>
We can define a valid path from a given start to end node in a graph as an ordered sequence of nodes [n<sub>1</sub>, n<sub>2</sub>, ... n<sub>k</sub>], where n<sub>1</sub> to n<sub>k</sub> are existing nodes in the graph and there is an edge from n<sub>i</sub> to n<sub>i+1</sub> for i = 1 to k - 1. In Figure 4, each edge is unweighted, so you can assume that each edge is length 1, and then the total distance traveled on the path is 4.

<img src='ProblemSet5/path.gif', width='25%'/>

Figure 4. Example of a path from start to end node.

Note that a graph can contain cycles. A cycle occurs in a graph if the path of nodes leads you back to a node that was already visited in the path. When building up possible paths, if you reach a cycle without knowing it, you could get stuck indefinitely by extending the path with the same nodes that have already been added to the path.

<img src='ProblemSet5/cycle.gif', width='25%'/>

Figure 5. Example of a cycle in a graph.

In our campus map problem, the **total distance traveled** on a path is equal to the sum of all total distances traveled between adjacent nodes on this path. Similarly, the **distance spent outdoors** on the path is equal to the sum of all distances spent outdoors on the edges in the path.

Depending on the number of nodes and edges in a graph, there can be multiple valid paths from one node to another, which may consist of varying distances. We define the **shortest path** between two nodes to be the path with the **least total distance traveled**. In our campus map problem, one way to find the shortest path from one building to another is to do exhaustive enumeration of all possible paths in the map and then select the shortest one.

How do we find a path in the graph? In the depth-first search algorithm, you try one route at a time while keeping track of routes tried so far. Work off the depth-first traversal algorithm covered in lecture to discover each of the nodes and their children nodes to build up possible paths. Note that you’ll have to adapt the algorithm to fit this problem. Read more about depth-first search [here](https://en.wikipedia.org/wiki/Depth-first_search).

Implement the function `bruteForceSearch(digraph, start, end, maxTotalDist, maxDistOutdoor)` so that for a given digraph, you return the shortest path, from the `start` building to `end` building, such that the total distance traveled is less than or equal to `maxTotalDist` and that the total distance spent outdoors is less than or equal to `maxDistOutdoor`.

For your own benefit, we encourage you to write a sentence describing what the optimization problem is in terms of what the function to minimize is and what the constraints are.

```python
# Problem 3: Brute Force Search
#
# State the optimization problem as a function to minimize
# and what the constraints are
#
```

Use the **depth-first** search approach from lecture to enumerate all possible paths from the start to end node on a given digraph. (Assume the start and end nodes are in the graph).

**Warning**: while you may choose to use DFS code given in lecture as a reference, you will have to adapt it to fit this problem!

Then select the paths that satisfy the constraint and from that group, pick the shortest path. Return this result as a list of nodes, [n<sub>1</sub>, n<sub>2</sub>, ... n<sub>k</sub>], where there exists an edge from n<sub>i</sub> to n<sub>i+1</sub> in the digraph, for all 1 <= i < k. If multiple paths are still found, then return any one of them. If no path can be found to satisfy these constraints, then raise a ValueError exception.

**Hint**: We suggest implementing one or more helper functions when writing bruteForceSearch. Consider first finding all valid paths that satisfy the max distance outdoors constraint, and then going through those paths and returning the shortest, rather than trying to fulfill both constraints at once.

```python
def bruteForceSearch(digraph, start, end, maxTotalDist, maxDistOutdoors):
    """
    Finds the shortest path from start to end using brute-force approach.
    The total distance traveled on the path must not exceed maxTotalDist,
    and the distance spent outdoor on this path must not exceed
    maxDistOutdoors.

    Parameters:
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    # TO DO
```

Paste your code for both `WeightedEdge` and `WeightedDigraph` in the box below. You may assume the grader has provided implementations for `Node`, `Edge`, and `Digraph`. Additionally paste your code for bruteForceSearch, and any helper functions, in this box.

In [ ]:
#
# Problem 3: Finding the Shortest Path using Brute Force Search
#
# State the optimization problem as a function to minimize
# and what the constraints are
#
# Use the depth-first search to find the shortest path
# Avoid cycles in the graph
#

In [153]:
def getAllPaths(digraph, start, end, path=[]):
    "Retrieve all paths from start node to end node"
    path = path + [start]
    
    if start == end:
        return [path]
    
    allPaths = []
    for node in digraph.childrenOf(start):
        if node not in path:
            newPaths = getAllPaths(digraph, node, end, path)
            for newPath in newPaths:
                allPaths.append(newPath)
    return allPaths
    
    
def getDistances(digraph, path):
    """Retrive the total and the outside distances."""
    total_d = 0
    out_d = 0
    
    for i in range(len(path) - 1):
        for node, dists in digraph.edges[path[i]]:
            totalDist, outdoorDist = dists
            if node == path[i + 1]:
                total_d += totalDist
                out_d += outdoorDist    
    return (total_d, out_d)


def bruteForceSearch(digraph, start, end, maxTotalDist, maxDistOutdoors):    
    """
    Finds the shortest path from start to end using brute-force approach.
    The total distance travelled on the path must not exceed maxTotalDist, and
    the distance spent outdoor on this path must not exceed maxDistOutdoors.

    Parameters: 
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by 
        a list of building numbers (in strings), [n_1, n_2, ..., n_k], 
        where there exists an edge from n_i to n_(i+1) in digraph, 
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    start = Node(start)
    end = Node(end)
    
    allPaths = getAllPaths(digraph, start, end, path=[])
        
    minPathDist = []
    minDist = float('inf')
    for path in allPaths:
        distance, outdoor = getDistances(digraph, path)
        if distance < minDist and distance <= maxTotalDist \
           and outdoor <= maxDistOutdoors:
            minDist = distance
            minPathDist = path
            
    if not minPathDist: raise ValueError
    return [el.getName() for el in minPathDist]

In [154]:
# Test code

g = WeightedDigraph()

na = Node('A')
nb = Node('B')
nc = Node('C')
nd = Node('D')
ne = Node('E')
nf = Node('F')
ng = Node('G')

g.addNode(na)
g.addNode(nb)
g.addNode(nc)
g.addNode(nd)
g.addNode(ne)
g.addNode(nf)
g.addNode(ng)

e1 = WeightedEdge(na, nb, 10, 2)
e2 = WeightedEdge(na, nc, 5, 2)
e3 = WeightedEdge(na, nd, 10, 5)
e4 = WeightedEdge(nb, ne, 2, 1)
e5 = WeightedEdge(nb, ng, 15, 3)
e6 = WeightedEdge(nd, nf, 5, 5)
e7 = WeightedEdge(nf, ng, 5, 0)


g.addEdge(e1)
g.addEdge(e2)
g.addEdge(e3)
g.addEdge(e4)
g.addEdge(e5)
g.addEdge(e6)
g.addEdge(e7)

#print g
minDist = bruteForceSearch(g, 'A', 'G', 100, 100)
print 'Minimum distance path:',minDist

Minimum distance path: ['A', 'D', 'F', 'G']


## Problem 4 - Optimized Method for Finding the Shortest Path

(20 points possible)<br>
Since enumerating all the paths is inefficient, let’s optimize our search algorithm for the shortest path. As you discover new children nodes in your depth-first search, you can keep track of the shortest path that so far that minimizes the distance traveled and minimizes the distance outdoors to fit the constraints.

If you come across a path that is longer than your shortest path found so far, then you know that this longer path cannot be your solution, so there is no point in continuing to traverse its children and discover all paths that contain this sub-path.

Implement the function `directedDFS(digraph, start, end, maxTotalDist, maxDistOutdoor)` that uses this optimized method to find the shortest overall path in a directed graph from start node to end node under the following constraints: the total distance travelled is less than or equal to `maxTotalDist`, and the total distance spent outdoors is less than or equal to `maxDistOutdoor`. If multiple paths are still found, then return any one of them. If no path can be found to satisfy these constraints, then raise a `ValueError` exception.

As with the previous problem, we suggest using one or more helper functions to implement `directedDFS` (see hint from above). In particular, is there any additional information you can pass or variable you can update that can help you reduce the number of nodes traversed?

Test your code by uncommenting the code at the bottom of ps5.py.
```python
def directedDFS(digraph, start, end, maxTotalDist, maxDistOutdoors):
    """
    Finds the shortest path from start to end using directed depth-first
    search approach. The total distance traveled on the path must not
    exceed maxTotalDist, and the distance spent outdoor on this path
    must not exceed maxDistOutdoors.

    Parameters:
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    # TO DO
```

Paste your code for both `WeightedEdge` and `WeightedDigraph` in this box. You may assume the grader has provided implementations for `Node`, `Edge`, and `Digraph`. Additionally paste your code for `directedDFS`, and any helper functions, in this box.

In [166]:
#
# Problem 4: Finding the Shorest Path using Optimized Search Method
#
def DFSMaxDist(digraph, start, end, maxTotalDist, maxDistOutdoors, path=[], bestPath=None):
    """Returns shortest path."""
    path = path + [start]
    if start == end:
        return path
    
    for node in digraph.childrenOf(start):
        if node not in path:
            dist, out = getDistances(digraph, path+[node])
            if dist <= maxTotalDist and out <= maxDistOutdoors:
                new_path = DFSMaxDist(digraph, node, end, maxTotalDist, maxDistOutdoors, path, bestPath)
                if new_path is not None:
                    distance, outdoor = getDistances(digraph, new_path)
                    if distance <= maxTotalDist and outdoor <= maxDistOutdoors:
                        bestPath = new_path
                        maxTotalDist = distance

    return bestPath


def directedDFS(digraph, start, end, maxTotalDist, maxDistOutdoors):
    """
    Finds the shortest path from start to end using directed depth-first.
    search approach. The total distance travelled on the path must not
    exceed maxTotalDist, and the distance spent outdoor on this path must
    not exceed maxDistOutdoors.

    Parameters: 
        digraph: instance of class Digraph or its subclass
        start, end: start & end building numbers (strings)
        maxTotalDist : maximum total distance on a path (integer)
        maxDistOutdoors: maximum distance spent outdoors on a path (integer)

    Assumes:
        start and end are numbers for existing buildings in graph

    Returns:
        The shortest-path from start to end, represented by 
        a list of building numbers (in strings), [n_1, n_2, ..., n_k], 
        where there exists an edge from n_i to n_(i+1) in digraph, 
        for all 1 <= i < k.

        If there exists no path that satisfies maxTotalDist and
        maxDistOutdoors constraints, then raises a ValueError.
    """
    start = Node(start)
    end = Node(end)

    bestPath = DFSMaxDist(digraph, start, end, maxTotalDist, maxDistOutdoors)

    if bestPath is None:
        raise ValueError

    return [node.getName() for node in bestPath]


In [167]:
# Test code

g = WeightedDigraph()

na = Node('A')
nb = Node('B')
nc = Node('C')
nd = Node('D')
ne = Node('E')
nf = Node('F')
ng = Node('G')
nh = Node('H')

g.addNode(na)
g.addNode(nb)
g.addNode(nc)
g.addNode(nd)
g.addNode(ne)
g.addNode(nf)
g.addNode(ng)
g.addNode(nh)

e1 = WeightedEdge(na, nb, 1, 2)
e2 = WeightedEdge(na, nc, 5, 2)
e3 = WeightedEdge(na, nd, 10, 5)
e4 = WeightedEdge(nb, ne, 2, 1)
e5 = WeightedEdge(nb, ng, 1, 3)
e6 = WeightedEdge(nd, nf, 10, 5)
e7 = WeightedEdge(nf, nh, 3, 0)
e8 = WeightedEdge(nh, ng, 3, 0)


g.addEdge(e1)
g.addEdge(e2)
g.addEdge(e3)
g.addEdge(e4)
g.addEdge(e5)
g.addEdge(e6)
g.addEdge(e7)
g.addEdge(e8)

#print g
minDist = directedDFS(g, 'A', 'G', 9, 100)
print 'Minimum distance path:',minDist

Minimum distance path: ['A', 'B', 'G']


In [ ]:
# Uncomment below when ready to test
#### NOTE! These tests may take a few minutes to run!! ####
# Test cases
mitMap = load_map(mitmap)
print isinstance(mitMap, Digraph)
print isinstance(mitMap, WeightedDigraph)
print 'nodes', mitMap.nodes
print 'edges', mitMap.edges

LARGE_DIST = 1000000

Test case 1
print "---------------"
print "Test case 1:"
print "Find the shortest-path from Building 32 to 56"
expectedPath1 = ['32', '56']
brutePath1 = bruteForceSearch(mitMap, '32', '56', LARGE_DIST, LARGE_DIST)
dfsPath1 = directedDFS(mitMap, '32', '56', LARGE_DIST, LARGE_DIST)
print "Expected: ", expectedPath1
print "Brute-force: ", brutePath1
print "DFS: ", dfsPath1
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath1 == brutePath1, expectedPath1 == dfsPath1)

Test case 2
print "---------------"
print "Test case 2:"
print "Find the shortest-path from Building 32 to 56 without going outdoors"
expectedPath2 = ['32', '36', '26', '16', '56']
brutePath2 = bruteForceSearch(mitMap, '32', '56', LARGE_DIST, 0)
dfsPath2 = directedDFS(mitMap, '32', '56', LARGE_DIST, 0)
print "Expected: ", expectedPath2
print "Brute-force: ", brutePath2
print "DFS: ", dfsPath2
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath2 == brutePath2, expectedPath2 == dfsPath2)

Test case 3
print "---------------"
print "Test case 3:"
print "Find the shortest-path from Building 2 to 9"
expectedPath3 = ['2', '3', '7', '9']
brutePath3 = bruteForceSearch(mitMap, '2', '9', LARGE_DIST, LARGE_DIST)
dfsPath3 = directedDFS(mitMap, '2', '9', LARGE_DIST, LARGE_DIST)
print "Expected: ", expectedPath3
print "Brute-force: ", brutePath3
print "DFS: ", dfsPath3
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath3 == brutePath3, expectedPath3 == dfsPath3)

Test case 4
print "---------------"
print "Test case 4:"
print "Find the shortest-path from Building 2 to 9 without going outdoors"
expectedPath4 = ['2', '4', '10', '13', '9']
brutePath4 = bruteForceSearch(mitMap, '2', '9', LARGE_DIST, 0)
dfsPath4 = directedDFS(mitMap, '2', '9', LARGE_DIST, 0)
print "Expected: ", expectedPath4
print "Brute-force: ", brutePath4
print "DFS: ", dfsPath4
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath4 == brutePath4, expectedPath4 == dfsPath4)

Test case 5
print "---------------"
print "Test case 5:"
print "Find the shortest-path from Building 1 to 32"
expectedPath5 = ['1', '4', '12', '32']
brutePath5 = bruteForceSearch(mitMap, '1', '32', LARGE_DIST, LARGE_DIST)
dfsPath5 = directedDFS(mitMap, '1', '32', LARGE_DIST, LARGE_DIST)
print "Expected: ", expectedPath5
print "Brute-force: ", brutePath5
print "DFS: ", dfsPath5
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath5 == brutePath5, expectedPath5 == dfsPath5)

Test case 6
print "---------------"
print "Test case 6:"
print "Find the shortest-path from Building 1 to 32 without going outdoors"
expectedPath6 = ['1', '3', '10', '4', '12', '24', '34', '36', '32']
brutePath6 = bruteForceSearch(mitMap, '1', '32', LARGE_DIST, 0)
dfsPath6 = directedDFS(mitMap, '1', '32', LARGE_DIST, 0)
print "Expected: ", expectedPath6
print "Brute-force: ", brutePath6
print "DFS: ", dfsPath6
print "Correct? BFS: {0}; DFS: {1}".format(expectedPath6 == brutePath6, expectedPath6 == dfsPath6)

Test case 7
print "---------------"
print "Test case 7:"
print "Find the shortest-path from Building 8 to 50 without going outdoors"
bruteRaisedErr = 'No'
dfsRaisedErr = 'No'
try:
    bruteForceSearch(mitMap, '8', '50', LARGE_DIST, 0)
except ValueError:
    bruteRaisedErr = 'Yes'
    
try:
    directedDFS(mitMap, '8', '50', LARGE_DIST, 0)
except ValueError:
    dfsRaisedErr = 'Yes'
    
print "Expected: No such path! Should throw a value error."
print "Did brute force search raise an error?", bruteRaisedErr
print "Did DFS search raise an error?", dfsRaisedErr

Test case 8
print "---------------"
print "Test case 8:"
print "Find the shortest-path from Building 10 to 32 without walking"
print "more than 100 meters in total"
bruteRaisedErr = 'No'
dfsRaisedErr = 'No'
try:
    bruteForceSearch(mitMap, '10', '32', 100, LARGE_DIST)
except ValueError:
    bruteRaisedErr = 'Yes'
    
try:
    directedDFS(mitMap, '10', '32', 100, LARGE_DIST)
except ValueError:
    dfsRaisedErr = 'Yes'
    
print "Expected: No such path! Should throw a value error."
print "Did brute force search raise an error?", bruteRaisedErr
print "Did DFS search raise an error?", dfsRaisedErr